In [1]:
# github에서 데이터 불러오기
!git clone https://github.com/chaylee1/DIC_education_at_KOREA_Univ.git

Cloning into 'DIC_education_at_KOREA_Univ'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 12 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.


In [2]:
import os, re, json
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

from gensim.models import FastText
from gensim.test.utils import datapath, get_tmpfile


In [3]:
def remove_symbols_stopwords(data):
    data['remove_symbols'] = [re.sub('[^A-Za-z0-9가-힣]', '', s) for s in data['asrresult']]
     
    remove_stopwords = []
    for s in tqdm(data['remove_symbols']):
        tokenized = ['<']
        for i in range(len(s)):
            tokenized.append(s[i])
        tokenized.append('>')
        
        s = ''
        for l in range(len(tokenized)):
            s += tokenized[l]
            
        remove_stopwords.append(s)
    
    data['remove_stopwords'] = pd.Series(remove_stopwords, index=data.index)
    return data


In [4]:
def train_FastText(data: pd.DataFrame,
                latent_dim: int,
                window_size: int,
                workers=-1,
                sg: int=0):
    model = FastText(vector_size=latent_dim,
                     window=window_size, min_count=1,
                     workers=workers, sg=sg)
    model.build_vocab(corpus_iterable=data['remove_stopwords'])
    model.train(
        epochs=300,
        corpus_iterable=data['remove_stopwords'],
        total_examples=10000)
    
    data_fasttext = []
    for i in list(data.index):
        t = data.loc[i, 'remove_stopwords']
        
        tmp_fasttest = []
        for i in range(len(t)-window_size+1):
            chr_split = model.wv[t[i:i+window_size]].tolist()
            tmp_fasttest.append(chr_split)
        
        data_fasttext.append(tmp_fasttest)
        
    return model, data_fasttext


In [5]:
# root_dir = '/workspace/HDD/PROJECT/2022_LG/TextClassification'
# data_dir = os.path.join(root_dir, 'data')
# data_dir = '/content/DIC_education_at_KOREA_Univ'
# data = pd.read_excel(os.path.join(data_dir, "data_pjt.xlsx"))

data = pd.read_excel('/content/DIC_education_at_KOREA_Univ/test.xlsx')
data



,asrresult
0,현재 시간뭐고
1,현재 시간뭐고
2,몇시야 몇시야
3,탈수
4,몇시야 4시 해봐
...,...
995,엑스맨
996,말 잘듣니
997,말 잘듣 냉장고는
998,비었어


In [6]:
data = remove_symbols_stopwords(data)

print(data)

model, data_fasttext = train_FastText(data,
                                      latent_dim=100,
                                      window_size=3,
                                      workers=-1,
                                      sg=0)

with open("/content/DIC_education_at_KOREA_Univ/fasttext.json", "w") as json_file:
    json.dump(data_fasttext, json_file)
json_file.close()

save_model_pth = "/content/DIC_education_at_KOREA_Univ/trained_fasttext.model"
model.save(save_model_pth)

model = FastText.load(save_model_pth)

100%|██████████| 1000/1000 [00:00<00:00, 153913.76it/s]


      asrresult remove_symbols remove_stopwords
0      현재 시간뭐고          현재시간뭐고         <현재시간뭐고>
1      현재 시간뭐고          현재시간뭐고         <현재시간뭐고>
2      몇시야 몇시야          몇시야몇시야         <몇시야몇시야>
3           탈수              탈수             <탈수>
4    몇시야 4시 해봐         몇시야4시해봐        <몇시야4시해봐>
..          ...            ...              ...
995        엑스맨             엑스맨            <엑스맨>
996      말 잘듣니            말잘듣니           <말잘듣니>
997  말 잘듣 냉장고는         말잘듣냉장고는        <말잘듣냉장고는>
998        비었어             비었어            <비었어>
999      말 잘듣니            말잘듣니           <말잘듣니>

[1000 rows x 3 columns]


TypeError: ignored